In [ ]:
import math
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from datetime import time
from numpy import mean, std
from prettytable import PrettyTable
from utils import (
    to_datetime,
    to_text,
    get_lective_data,
    get_lective_dates,
    next_days,
    find_hour_day,
    plot_and_save
)

The code below are a little messy...

In [ ]:
#Let's define some 'global' variables

keys = [
    'eCO2 in ppm (AMS CCS811 - eCO2)',
    'TVOC in ppb (AMS CCS811 - TVOC)',
    'PM 1 in ug/m3 (PMS5003_AVG-PM1)',
    'PM 10 in ug/m3 (PMS5003_AVG-PM10)',
    'PM 2.5 in ug/m3 (PMS5003_AVG-PM2.5)'
]

standards = [
    1000,
    150, # 0.3/2 * 1000 -> `very good` limit
    None,
    50,
    25
]

names = [
    'eCO2',
    'TVOC',
    'PM1',
    'PM10',
    'PM25'
]


horaris = [ (time(s_h, s_m), time(e_h, e_m)) for s_h, s_m, e_h, e_m in 
    [
        ( 8,30,  9,15),
        ( 9,15, 10, 0),
        (10, 0, 10,45),
        (10,45, 11,30),
        (11,30, 12, 0),
        (12, 0, 12,45),
        (12,45, 13,30),
        (13,30, 14,15),
        (14,15, 15, 0)
    ]
]


dies_lectius = list(range(5))

In [ ]:
# ------------OUTDOOR data-------------------------------------

with open('exterior.csv', 'r', encoding='UTF-8') as f:
    df = pd.read_csv(f)
    
days_ex = {}
tmp = df
gen = next_days('2020-10-19 00:00', -1) # extract reversely
for i in range(21):
    day = to_text(next(gen))
    days_ex[day[:10]] = tmp[tmp.timestamp >= day]
    tmp = tmp[tmp.timestamp < day]

# -------------INDOOR data-----------------------------------

with open('interior.csv', 'r', encoding='UTF-8') as f:
    df = pd.read_csv(f)
    
days_in = {}
tmp = df
gen = next_days('2020-10-19 00:00', -1) # extract reversely
for i in range(21):
    day = to_text(next(gen))
    days_in[day[:10]] = tmp[tmp.timestamp >= day]
    tmp = tmp[tmp.timestamp < day]
    
# -------------------------------------------------------------
    
# The keys of days are kept inversely: starting from the last day
# So we have to use its inverse to plot the data

k_inv = []
for k in days:
    k_inv.append(k)
    
k_inv = k_inv[::-1]

In [ ]:
# Plot the graphics to compare the indoor and outdoor data
# for each week.

# Notice that in order to synchronize the data
# and to label differents days and weeks,
# this may be a little difficult for further debugging...

for num, key in enumerate(keys): # for every air parameter...
    # One plot for week, and there are three
    fig, ax = plt.subplots(3)
    fig.set_size_inches(20,25)
    
    # Find the ylim. Be careful that the data may contain nan
    y_lim = standards[num] if standards[num] is not None else 0
    for k in days_in:
        y_lim = max([*[e for e in list(days[k][key]) if not math.isnan(e)], y_lim])
    for k in days_ex:
        y_lim = max([*[e for e in list(days[k][key]) if not math.isnan(e)], y_lim])
        
     # to be the first position while ploting
    p_in = len(days_in['2020-09-29'][key]) - len(days_in['2020-09-28'][key])
    p_ex = len(days_ex['2020-09-29'][key]) - len(days_ex['2020-09-28'][key])
    positions = [] # array to keep the position where we should put the xticks
    
    for i, k in enumerate(k_inv):
        data_in = days_in[k][key]
        data_ex = days_ex[k][key]
        size_in = len(data_in)
        size_ex = len(data_ex)
        ax[i//7].plot(range(p_in,p_in+size_in), data_in, color='tab:blue')
        ax[i//7].plot(range(p_ex,p_ex+size_ex), data_ex, color='tab:red')
        positions.append(p_in if i != 0 else 0)
        p_in += size_in
        p_ex += size_ex
        if (i+1)%7 != 0: # do not plot a vertical line at the end
            ax[i//7].vlines(p_in, 0, y_lim, color='gray', linewidth=2)
            ax[i//7].vlines(p_ex, 0, y_lim, color='gray', linewidth=2)
    
        if (i+1)%7 == 0 and i != 0: # now, let's plot the week
            ax[i//(7+1)].hlines(standards[num], 0, 10000, color='black', linewidth=3)
            plt.sca(ax[i//(7+1)])
            
            handles = [
                mpatches.Patch(color='black',label=str(standards[num])),
                mpatches.Patch(color='tab:blue',label='interior'),
                mpatches.Patch(color='tab:red', label='exterior')
                      ]
            plt.legend(handles=handles, fontsize=20)
            plt.xlim(-500, 10500)  # empirical values
            plt.xticks(positions, ['DL', 'DM', 'DC', 'DJ', 'DV', 'DS', 'DM'])
            # reassign the variables
            positions = []
            tickes = []
            p_in = 0
            p_ex = 0

    for j in range(3): # change the font size
        for item in ([ax[j].title, ax[j].xaxis.label, ax[j].yaxis.label]
                + ax[j].get_xticklabels() + ax[j].get_yticklabels()):
            item.set_fontsize(20)
    
    fig.suptitle(key, fontsize=20)
    plt.savefig('Comp_'+str(names[num])+'.png', dpi=200)
    plt.clf()

In [ ]:
# The function below returns directly a dictionary of shape
# d = {
#   'eCO2 in ppm (AMS CCS811 - eCO2)': 
#     [[[], [], [], [], []],
#      [[], [], [], [], []],
#      [[], [], [], [], []],
#      [[], [], [], [], []],
#      [[], [], [], [], []],
#      [[], [], [], [], []],
#      [[], [], [], [], []],
#      [[], [], [], [], []]],
#   'X':[...], 
#   'Y':[...]
#     }

#

def get_timetable(path):
    labels, data = get_lective_data(path)
    dates = get_lective_dates(data[0][0], data[-1][0])
    time_table = dict.fromkeys(keys)

    for key in keys:
        time_table[key] = []
        for i in range(len(horaris)):
            time_table[key].append([])
            for j in range(len(dies_lectius)):
                time_table[key][i].append([])

    for row in data:
        pack = find_hour_day(row[0], horaris, dies_lectius)
        # print('date:', row[0], 'pack', pack)
        if pack is None:
            continue
        hora, dia = pack
        for i, number in enumerate([5, 6, 8, 10, 11]):# column of the parameters
            if not math.isnan(row[number]):
                time_table[keys[i]][hora][dia].append(row[number]) # humidity
            
    # we have filled the data to the time_table

    for key in keys:
        for i in range(len(horaris)):
            for j in range(len(dies_lectius)):
                time_table[key][i][j] = round(mean(time_table[key][i][j]), 1)

    return time_table

In [ ]:
with open('exterior.csv', 'r', encoding='UTF-8') as f:
    df = pd.read_csv(f)

ex_week1 = df[df.timestamp < '2020-10-05']
tmp = df[df.timestamp >= '2020-10-05']
ex_week2 = tmp[tmp.timestamp < '2020-10-12']
ex_week3 = tmp[tmp.timestamp >= '2020-10-12']

ex_week1.to_csv('ex_week1.csv', index=False)
ex_week2.to_csv('ex_week2.csv', index=False)
ex_week3.to_csv('ex_week3.csv', index=False)

In [ ]:
with open('interior.csv', 'r', encoding='UTF-8') as f:
    df = pd.read_csv(f)

in_week1 = df[df.timestamp < '2020-10-05']
tmp = df[df.timestamp >= '2020-10-05']
in_week2 = tmp[tmp.timestamp < '2020-10-12']
in_week3 = tmp[tmp.timestamp >= '2020-10-12']

in_week1.to_csv('in_week1.csv', index=False)
in_week2.to_csv('in_week2.csv', index=False)
in_week3.to_csv('in_week3.csv', index=False)

In [ ]:
for situation in ('in_', 'ex_'):
    for week in ['week'+str(i) for i in range(1,4)]:
        path = situation+week+'.csv'
        time_table = get_timetable(path)
        print('\n--------------------------', path, '--------------------------')
        for j,k in enumerate(keys):
            table = PrettyTable()
            table.field_names = [names[j], 'Dilluns', 'Dimarts', 'Dimecres', 'Dijous', 'Divendres']
            for i, lesson in enumerate(horaris):
                start, end = [to_text(l, '%H:%M') for l in lesson]
                table.add_row([start+'-'+end, *(time_table[k][i])])
            print(table)